In [19]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt
import dlib
from imutils import face_utils
import joblib
import pickle as pk
import keras
import itertools

In [3]:
face_classifier = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
#model_state = torch.load("./emotion_detection_model_state.pth")
class_labels = ["Angry", "Happy", "Neutral", "Sad", "Suprise"]#["Angry","Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Suprise"]

In [4]:
model = keras.models.load_model("mlp_dataset_60%_5emotions.h5") #pk.load(open('models/MLP/mlp2_62%_8emotions.h5', 'rb'))

In [25]:
with open("model6_nose_boost955", "rb") as f:
    model_boost = pk.load(f)
    f.close()

In [26]:
model_boost

GradientBoostingClassifier(learning_rate=0.0005, loss='deviance', max_depth=20,
                           max_features='sqrt', min_samples_leaf=2,
                           min_samples_split=4, n_estimators=1000,
                           subsample=0.6)

In [5]:
#predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [6]:
img_shape = 48 #96

In [7]:
# std = np.load('std.npy')
# mean = np.load('mean.npy')
with open("vars/MediaPipe/scaler-train-dataset-184.pkl","rb") as f:
    scaler = pk.load(f)
    f.close()

In [8]:
# with open("pca.pkl", "rb") as f:
#     pca = pk.load(f)
#     f.close()

In [9]:
print("Scaling std", scaler.var_, "\n\nmean", scaler.mean_)

Scaling std [0.00324147 0.00367071 0.00404544 0.00442004 0.00467529 0.00412794
 0.0050524  0.00274372 0.00289259 0.00486056 0.00369532 0.00329358
 0.00455896 0.00307183 0.0029126  0.00280061 0.0050166  0.00580539
 0.00534945 0.00475048 0.00289437 0.00525387 0.00308015 0.00421326
 0.00367464 0.00558246 0.00334566 0.00311453 0.00296689 0.00351064
 0.00405499 0.00453406 0.00657046 0.00407183 0.00453578 0.0073784
 0.00561993 0.00365897 0.00642162 0.00500043 0.00577628 0.00298736
 0.00434056 0.00482589 0.00827847 0.00611674 0.00378975 0.00726693
 0.00547174 0.00644452 0.00308064 0.00729349 0.00101503 0.00102918
 0.00142235 0.00190726 0.00135449 0.00275431 0.00313945 0.00264322
 0.00300136 0.00295355 0.00243484 0.00221456 0.00292045 0.00106916
 0.00148189 0.00199044 0.00264009 0.00270829 0.00301183 0.00181139
 0.00151151 0.00136624 0.00254194 0.00307385 0.00271925 0.0026133
 0.00303124 0.00231186 0.00287891 0.00264924 0.00280804 0.00191118
 0.00159217 0.00141379 0.00311818 0.00271305 0.00263

In [10]:
def euc(a, b):
    return np.sqrt( (a[0]-b[0])*(a[0]-b[0]) + (a[1]-b[1])*(a[1]-b[1]) )

In [11]:
def euc2d(a, b):
    return np.sqrt( (a[0]-b[0])*(a[0]-b[0]) + (a[1]-b[1])*(a[1]-b[1]) )

In [12]:
def euc3d(a, b):
    return np.sqrt( (a[0]-b[0])*(a[0]-b[0]) + (a[1]-b[1])*(a[1]-b[1]) + (a[2]-b[2])*(a[2]-b[2]) )

In [13]:
def calc_dist(shape):
    V1 = (euc(shape[1],shape[5]) + euc(shape[2],shape[4])) / 2
    V2 = euc(shape[0], shape[3])            
    V3 = (euc(shape[7],shape[11]) + euc(shape[8],shape[10])) / 2
    V4 = euc(shape[6], shape[9])            
    V5 = euc(shape[12], shape[14])
    V6 = euc(shape[15], shape[17])            
    V7 = euc(shape[19], shape[25])
    V8 = (euc(shape[1],shape[13]) + euc(shape[2],shape[13])) / 2            
    V9 = (euc(shape[7],shape[16]) + euc(shape[8],shape[16])) / 2
    V10 = euc(shape[18], shape[22])            
    V11 = (euc(shape[5],shape[19]) + euc(shape[4],shape[19])) / 2
    V12 = (euc(shape[11],shape[25]) + euc(shape[10],shape[25])) / 2            
    V13 = euc(shape[3], shape[14])
    V14 = euc(shape[6], shape[15])            
    V15 = euc(shape[20], shape[30])
    V16 = euc(shape[21], shape[29])            
    V17 = euc(shape[22], shape[28])
    V18 = euc(shape[23], shape[27])            
    V19 = euc(shape[24], shape[26])
    V20 = euc(shape[19], shape[18])            
    V21 = euc(shape[25], shape[18])
    V22 = (euc(shape[33],shape[34]) + euc(shape[35],shape[38]) + euc(shape[36],shape[37])) / 3            
    V23 = euc(shape[3],shape[19])
    V24 = euc(shape[6],shape[20])

    
    return V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24

In [14]:
import mediapipe as mp

In [15]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [16]:
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5)

In [17]:
# cap = cv2.VideoCapture(0)

# while True:
#     # Grab a single frame of video
#     ret, frame = cap.read()
#     frame = cv2.flip(frame, 1)
#     labels = []
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)

#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         roi_gray = gray[y : y + h, x : x + w]
#         roi_gray = cv2.resize(roi_gray, (img_shape, img_shape), interpolation=cv2.INTER_AREA)
#         roi_gray = cv2.GaussianBlur(roi_gray, (3,3), cv2.BORDER_DEFAULT)

#         if np.sum([roi_gray]) != 0:
            
#             landmarks = predictor(roi_gray, dlib.rectangle(0, 0, img_shape, img_shape))
#             shape = face_utils.shape_to_np(landmarks)
#             shape = shape[[36, 37, 38, 39, 40, 41] + [42, 43, 44, 45, 46, 47] + [17, 19, 21] + [22, 24, 26] + [30, 31, 35] + [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 67, 62, 63, 65, 66]]
            
#             pred_i = model.predict((np.array([calc_dist(shape)]) - mean) / std)
#             try:
#                 cv2.putText(
#                     frame,
#                     f"{class_labels[pred_i[0]]}",
#                     (20, 60),
#                     cv2.FONT_HERSHEY_COMPLEX,
#                     2,
#                     (0, 255, 0),
#                     3,
#                 )
#             except:
#                 continue
#         else:
#             cv2.putText(
#                 frame,
#                 "No Face Found",
#                 (20, 60),
#                 cv2.FONT_HERSHEY_COMPLEX,
#                 2,
#                 (0, 255, 0),
#                 3,
#             )

#     cv2.imshow("Emotion Detector", frame)

#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [20]:
LEFT_EYE = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
RIGHT_EYE = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
LEFT_EYEBROW = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYEBROW)))
RIGHT_EYEBROW = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYEBROW)))
LIPS = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
CONTOURS = list(set(itertools.chain(*mp_face_mesh.FACEMESH_CONTOURS)))
OTHER = [1] #0, 1, 6, 8, 10

In [30]:
cap = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    labels = []
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        
        cv2.rectangle(rgb, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi = rgb[y : y + h, x : x + w]
        roi = cv2.resize(roi, (img_shape, img_shape), interpolation=cv2.INTER_AREA)
        roi = cv2.GaussianBlur(roi, (3,3), cv2.BORDER_DEFAULT)
        
        results = face_mesh.process(roi)
        
        if results.multi_face_landmarks:
        
            shape = [(lmk.x, lmk.y, lmk.z) for lmk in results.multi_face_landmarks[0].landmark]
            shape = np.array(shape)
            nose = shape[1]
            shape = shape[LEFT_EYE + RIGHT_EYE + LEFT_EYEBROW + RIGHT_EYEBROW + LIPS]

            #distances2d = distance.cdist(shape[1], shape, 'euclidean')

            distances2d = [round(euc2d(nose, x), 6) for x in shape]
            distances3d = [round(euc3d(nose, x), 6) for x in shape]

            #df.append(pd.DataFrame([np.append(distances.flatten(), i)], columns=columns))

            values = np.array(distances2d + distances3d).reshape(1, -1)
            
            values = scaler.transform(values)
            
            pred = model.predict(values, verbose=0)
            
            pred1_i = model_boost.predict(values)[0]
            
            pred_i = np.argmax(pred, axis=1)[0]
            
            #print(pred_i)
            
            try:
                cv2.putText(
                    frame,
                    f"{class_labels[pred_i]}",
                    (20, 60),
                    cv2.FONT_HERSHEY_COMPLEX,
                    2,
                    (0, 255, 0),
                    3,
                )
            except:
                continue
        else:
            cv2.putText(
                frame,
                "No Face Found",
                (20, 60),
                cv2.FONT_HERSHEY_COMPLEX,
                2,
                (0, 255, 0),
                3,
            )

    cv2.imshow("Emotion Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.wa

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.wa

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.wa

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
 

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.wa

C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
C:\Users\Nur\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.wa